In [109]:
from pathlib import Path
import os
import pandas as pd


In [110]:
dir_path = Path(os.path.join('..','tmp'))
dataset_path = Path(os.path.join('..','dataset','objects'))
data_path = dataset_path.joinpath('data.csv')

### CVAT download dataset

In [111]:
import requests
import shutil
import sys
from glob import glob
import cv2
sys.path.insert(0, "..")
from zipfile import ZipFile

from io import BytesIO

In [112]:
URL = 'http://10.72.125.105:8080'
API_URL = '%s/api/v1' % URL
LOGIN = 'ml'
PASSWORD = 'NuAzu8mO'
PROJECT = 'Liza'

def get_session(url, username, password):
    session = requests.Session()
    response=session.post(url, data={'username':username,'password':password})
    response.raise_for_status()
    if 'csrftoken' in response.cookies:
        session.headers['X-CSRFToken'] = response.cookies['csrftoken']
    return session

def get_request(self, method, query=None):
    url = "%s/%s" % (API_URL,method)
    if ~(query is None):
        url+='?%s'%query
    request = self.get(url)
    if request.headers['Content-Type'] == 'application/json':
        return request.json()
    else:
        return request

In [113]:
url = '%s/auth/login' % API_URL
session = get_session(url, LOGIN, PASSWORD)
projects = get_request(session,'projects','?search=%s' % (PROJECT))
project= projects['results'][0]

In [114]:
labels=pd.DataFrame(project['labels'])

In [115]:
def makePath(path):
    if path.exists() and path.is_dir():
        shutil.rmtree(path)
    os.mkdir(path)

In [116]:
makePath(dir_path)
makePath(dataset_path)

for task in project['tasks']:
    task_id = task['id']
    chunk_size = task['data_chunk_size']
    annotations = get_request(session,'tasks/%s/annotations' % (task_id))
    shapes = pd.DataFrame(annotations['shapes'])
    chunk_number=0

    for chunk_n in range((shapes.frame.max() // chunk_size) + 1):
        start_chunk = chunk_n * chunk_size
        chunk_shapes = shapes[(shapes.frame >= start_chunk) & (shapes.frame < start_chunk+chunk_size)]

        if len(chunk_shapes) > 0:
            data=get_request(session,'tasks/%s/data' % task_id, 'type=chunk&number=%s&quality=original' % chunk_number)
            zipfile= ZipFile(BytesIO(data.content))
            zipfile.extractall(dir_path)

            for frame in chunk_shapes.frame.unique():
                filename = '%s.*' % str(1000000 + (frame-start_chunk))[1:]
                filename = glob(str(dir_path.joinpath(filename)))[0]
                image_id = '%s_%s' % (task_id,frame)
                
                new_filename = ('%s.%s' % (image_id, filename.split('.')[-1])).lower()
                shapes.loc[shapes['frame']==frame, 'image_path'] = new_filename
                shutil.move(dir_path.joinpath(filename),dataset_path.joinpath(new_filename))

shutil.rmtree(dir_path)

In [117]:
shapes['label']=shapes.apply(lambda x: labels[labels['id']==x.label_id].index[0], axis=1)

boxes=pd.DataFrame(shapes['points'].to_list(), columns=['x1', 'y1', 'x2', 'y2'])
shapes=pd.concat([shapes,boxes], axis=1)

images_path = dataset_path.joinpath('images')
labels_path = dataset_path.joinpath('labels')
makePath(images_path)
makePath(labels_path)

In [118]:
for image in shapes.image_path.unique():
    im=cv2.imread(str(dataset_path.joinpath(image)))
    shutil.move(dataset_path.joinpath(image),images_path.joinpath(image))
    coords = shapes[shapes['image_path']==image][['label','x1', 'y1', 'x2', 'y2']]
    width = im.shape[1]
    height = im.shape[0]
    
    txt_path = labels_path.joinpath('%s.txt' % image.split('.')[0])
    coords.apply(lambda x: pd.Series([
        x.label,
        ((x.x2-x.x1)/2)/width, 
        ((x.y2-x.y1)/2)/height, 
        (x.x2-x.x1)/width, 
        ((x.y2-x.y1)/2)/height])
    , axis=1).to_csv(txt_path, sep=' ', index=None, header=None)

In [120]:
labels.name.unique()

array(['drons', 'auto', 'swimmer', 'horse', 'hug', 'sherp', 'aero', 'dog'],
      dtype=object)

In [124]:
!python3 ../yolov5/train.py --img 640 --batch 16 --epochs 3 --data liza.yaml --weights yolov5s.pt

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5s.pt, cfg=, data=liza.yaml, hyp=../yolov5/data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=../yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-261-g19f33cb Python-3.8.10 torch-1.10.0+cu102 CUDA:0 (GRID P40-12Q, 12288MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.9